In [1]:
import tensorflow as tf
import os
from keras.models import Model
from keras.layers import Dense, Flatten
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
os.chdir("../")

In [ ]:
# Load the pre-trained model
model = tf.keras.models.load_model('models/model.h5')

# Define parameters
batch_size = 32
epochs = 25

In [4]:
# Image data generators for training and validation
train_gen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

validation_gen = ImageDataGenerator(rescale=1./255)

# Ensure the correct paths for training and validation directories
train_generator = train_gen.flow_from_directory(
    'artifacts/training',  # Make sure this path is correct
    target_size=(160, 160), 
    batch_size=batch_size,
    class_mode='sparse',  # Use 'sparse' for sparse categorical crossentropy
    color_mode='rgb',
    shuffle=True
)

val_generator = validation_gen.flow_from_directory(
    'artifacts/training',  # Update to the correct validation path
    target_size=(160, 160),
    batch_size=batch_size,
    class_mode='sparse',  # Use 'sparse' for sparse categorical crossentropy
    color_mode='rgb',
    shuffle=False
)

Found 200 images belonging to 4 classes.


Found 200 images belonging to 4 classes.


In [ ]:
new_model = Model(inputs=model.input, outputs=model.layers[-4].output)

# Freeze all layers
for layer in new_model.layers:
    layer.trainable = False

In [9]:
x = Dense(220, activation='relu')(new_model.output)
x = Dense(128, activation='relu')(x)
output = Dense(4, activation='softmax')(x)  # Assuming 4 classes

In [ ]:
final_model = Model(inputs=new_model.input, outputs=output)

# Compile the model
final_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Fit the model
history = final_model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=epochs,
    validation_data=val_generator,
    validation_steps=val_generator.samples // batch_size
)

In [ ]:
import matplotlib.pyplot as plt

# Assuming `history` is the History object returned by model.fit()
# Plot training & validation accuracy values
plt.figure(figsize=(12, 5))

# Plotting accuracy
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(loc='lower right')

# Plotting loss
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(loc='upper right')

# Show the plots
plt.tight_layout()
plt.show()